In [4]:
import pandas as pd
import numpy as np 
import os 
import glob
import re
from pigeon import annotate
from IPython.display import display, Image

from modules.heic_to_jpg import convert_heic_to_jpg_from_directory
from modules.face_detection import face_detection_from_source_folder
from modules.data_processing import train_test_split_labeled_data
from modules.data_processing import copy_images_to_new_folder
import modules.face_recog


2021-11-24 21:57:57.479203: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/api/keras/optimizers


AlreadyExistsError: Another metric with the same name already exists.

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np 

names = []
# Iterate over test images
for image_file in os.listdir("data/test/seth"):
    if image_file[len(image_file)-1] != 'g':
        continue
    full_file_path = os.path.join("data/test/seth", image_file)

    print("Looking for faces in {}".format(image_file))

    # Find all people in the image using a trained classifier model
    # Note: You can pass in either a classifier file name or a classifier model instance
    predictions = face_recog.predict(full_file_path, cropped_face = True, model_path="models/trained_knn_model.clf")

    # Print results on the console
    for name, (top, right, bottom, left) in predictions:
        print("- Found {} at ({}, {})".format(name, left, top))

    # Display results overlaid on an image
    # face_recog.show_prediction_labels_on_image(os.path.join("data/test/claire", image_file), predictions)
    img_path = os.path.join("data/test/seth", image_file)
    
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)
    for name, (top, right, bottom, left) in predictions:
        names.append(name)
        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # There's a bug in Pillow where it blows up with non-UTF-8 text
        # when using the default bitmap font
        name = name.encode("UTF-8")

        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))
        
    del draw

    # Display the resulting image
    plt.imshow(np.array(pil_image))
    plt.show()
    
